In [4]:
import pandas as pd
import geopy.distance
import math
import random

path = open("./Mappa-delle-antenne-in-Italia.csv", encoding="utf8", errors='ignore')

in_data = pd.read_csv(path, sep = ";")

interesting_data = in_data[in_data["Comune"] == "Bergamo"].iloc[:, 7:9]

interesting_data.to_excel('./BaseStationLocation.xlsx', index = False)

#  Find three locations for MEC Hosts, greedy center selction

In [188]:
def maxds(s,c):
    """
    Method used to fin the max distances from the set of BSS to the center of the centers 
    """
    maxel = random.sample(s, 1)                        # start from random
    maxd = geopy.distance.distance(maxel, c).km
    for i in s:
        d = geopy.distance.distance(i, c).km
        if maxd < d:
            maxd = d
            maxel = i
    return maxel

def center_geo(geolocations):
    """
    Find center lat, lon 
    """
    x = 0
    y = 0
    z = 0

    for lat, lon in geolocations:
        lat = float(lat)
        lon = float(lon)
        x += math.cos(lat) * math.cos(lon)
        y += math.cos(lat) * math.sin(lon)
        z += math.sin(lat)

    x = float(x / len(geolocations))
    y = float(y / len(geolocations))
    z = float(z / len(geolocations))

    return (math.atan2(z, math.sqrt(x * x + y * y)), math.atan2(y, x))

def find_centers(input_data, n):
    """
    Find n centers of the data center selection implementation greedy, starting from a center point
    """
    start = input_data.iloc[4].values
    temp = set(tuple(i) for i in input_data.values)
    centers = [start]
    c = start
    temp.remove(tuple(start))
    
    for i in range(n - 1):
        el = maxds(temp,c)
        centers.append(el)
        temp.remove(el)
        c = center_geo(centers)
    return centers


In [189]:
centers = find_centers(interesting_data, 3)
centers

[array([ 9.686109 , 45.6898246]),
 (9.6516641, 45.6822899),
 (9.6962931, 45.71185069999989)]

In [190]:
pd.DataFrame(centers, columns = ['longitude', 'latitude']).to_excel('./MecHostLocation.xlsx', index = False)